# Deck of Cards
- Design the data structures for a **generic deck of cards**.
- Explain how you would subclass the data structures to implement blackjack.

In [82]:
from enum import Enum
from itertools import product
from operator import attrgetter
import random
from collections import deque
from collections.abc import Iterable


class CardSuit:
    pass


class Card:
    # TODO: Add docstrings
    def __init__(self, suit: CardSuit, value: int, name: str = None):
        self.suit = suit
        self.value = value
        self.name = name  # Used in the French deck

    def __repr__(self):
        card_value = self.name if self.name is not None else str(self.value)
        return f"{card_value}-{self.suit.name}"
    
    def __str__(self):
        return self.__repr__()


class FrenchDeckSuits(Enum):
    SPADES = 0
    HEARTS = 1
    CLUBS = 2
    DIAMONDS = 3


class Deck:
    def __init__(self):
        self.cards = deque()

    @property
    def n_cards(self):
        return len(self.cards)

    def shuffle(self):
        random.shuffle(self.cards)
        return self

    def draw_from_the_top(self):
        return self.cards.pop()
    
    def add_cards(self, cards: Iterable[Card]):
        self.cards.extend(cards)


class FrenchDeck(Deck):
    CARD_NAMES = {1: "Ace", 11: "Jack", 12: "Queen", 13: "King"}

    def __init__(self):
        super().__init__()
        self.build_deck()

    def build_deck(self):
        for suit, value in product(FrenchDeckSuits, range(1, 14)):
            card = Card(suit, value, FrenchDeck.CARD_NAMES.get(value))
            self.cards.append(card)


class BlackJackPlayer:
    def __init__(self, name: str, ambitious: bool = False):
        self.hand: list[Card] = []
        self.name = name
        self.ambitious = ambitious

    def __repr__(self):
        return f"{self.name}({self.hand})"
    
    def say(self, msg: str):
        print(f"{self.name}: {msg}")

    @property
    def wants_to_draw(self):
        folding_threshold = 20 if self.ambitious else 17
        return self.hand_total <= folding_threshold

    def add_card(self, card: Card):
        print(f"{self.name}: adding card {card}")
        self.hand.append(card)

    @property
    def hand_total(self) -> int:
        total = sum(min(card.value, 10) for card in self.hand)
        return total
    

class Crupier(BlackJackPlayer):
    @property
    def wants_to_draw(self):
        return self.hand_total <= 16


class BlackJackGame:
    # TODO:
    # - Consider bets
    # - Aces might be 11 or 1, the player decides
    # - Special case Ace + 10 is called Blackjack

    def __init__(self, n_players: int = 1, n_decks: int = 1):
        self.deck = FrenchDeck().shuffle()
        self.n_players = n_players
        self.players = [BlackJackPlayer("Crupier")]
        self.players += [BlackJackPlayer(f"Player{i}") for i in range(n_players)]

    def draw(self) -> Card:
        return self.deck.draw_from_the_top()

    def draw_for_player(self, player_i: int = 0):
        player = self.players[player_i]
        player.add_card(self.draw())

    def deal_for_players(self):
        for player in self.players:
            while player.wants_to_draw and player.hand_total <= 21:
                player.add_card(self.deck.draw_from_the_top())
            player.say(f"My total is {player.hand_total}\n---")

    def decide_winner(self) -> BlackJackPlayer:
        players_left = [p for p in self.players if p.hand_total <= 21]
        winner = max(players_left, key=attrgetter("hand_total"))
        print(f"The winner is {winner}")
        return winner

    def __repr__(self):
        return f"BlackJack({self.players})"


####

game = BlackJackGame(n_players=2, n_decks=3)
game.players[0].ambitious = True
game.players[1].ambitious = False
display(game)
game.deal_for_players()
game.decide_winner()
game
display(game)

BlackJack([Crupier([]), Player0([]), Player1([])])

Crupier: adding card 10-CLUBS
Crupier: adding card 9-DIAMONDS
Crupier: adding card 2-CLUBS
Crupier: My total is 21
---
Player0: adding card 9-SPADES
Player0: adding card Ace-CLUBS
Player0: adding card 6-SPADES
Player0: adding card 9-HEARTS
Player0: My total is 25
---
Player1: adding card 8-HEARTS
Player1: adding card Jack-SPADES
Player1: My total is 18
---
The winner is Crupier([10-CLUBS, 9-DIAMONDS, 2-CLUBS])


BlackJack([Crupier([10-CLUBS, 9-DIAMONDS, 2-CLUBS]), Player0([9-SPADES, Ace-CLUBS, 6-SPADES, 9-HEARTS]), Player1([8-HEARTS, Jack-SPADES])])